# STAC
Let's use EOReader to create [SpatioTemporal Asset Catalog (STAC)](https://stacspec.org/) items.

<div class="alert alert-info">
  
   <strong>Note:</strong> This is experimental for now, use it at your own risk !
    
</div>

<div class="alert alert-warning">
  
<strong>Warning:</strong>
You will need to install <a href="https://pystac.readthedocs.io/en/stable/"><code>pystac[validation]</code></a>,
<a href="https://python-visualization.github.io/folium/"><code>folium</code></a>
and <a href="https://eodag.readthedocs.io/en/stable/"><code>eodag</code></a> (version != 2.6.0) to run this notebook
    
</div>

## Imports

In [1]:
# Imports
import os

import pystac
import geopandas as gpd
from tempfile import TemporaryDirectory
from shapely.geometry import mapping

from eodag import setup_logging
from eodag.api.core import EODataAccessGateway

from eoreader.reader import Reader

## Create logger

In [2]:
# Create logger
import logging
from sertit import logs

logger = logging.getLogger("eoreader")
logs.init_logger(logger)

## Linking some data 

Let's take 3 products covering approximately the same area (over DAX city in France):
- One Landsat-8 OLI-TIRS collection 2
- One Landsat-5 TM collection 2
- One Sentinel-2 L1C

In [3]:
prod_folder = os.path.join("/home", "prods")
paths = [
    # Landsat-8 OLI-TIRS collection 2
    os.path.join(prod_folder, "LANDSATS_COL2", "LC08_L1TP_200030_20201220_20210310_02_T1.tar"),
    # Landsat-5 TM collection 2    
    os.path.join(prod_folder, "LANDSATS_COL2", "LT05_L1TP_200030_20111110_20200820_02_T1.tar"),
    # Sentinel-2 L2A
    os.path.join(prod_folder, "S2", "PB 02.07+", "S2A_MSIL1C_20191215T110441_N0208_R094_T30TXP_20191215T114155.SAFE"),
]

## Create STAC catalog
Create a STAC catalog and add 3 STAC items to it.

In [4]:
# Create the reader
reader = Reader()

# Work in a temporary directory
tmp = TemporaryDirectory()

In [5]:
# Create STAC catalog
catalog_path = os.path.join(tmp.name, "catalog.json")
catalog = pystac.Catalog(
    id='SERTIT_101',
    description="SERTIT's Catalog",
    title='SERTIT Catalog',
    href=catalog_path
)

In [6]:
# Add all the products into the STAC catalog
for path in paths:
    logger.info(f"*** {os.path.basename(path)} ***")

    # Open the product
    prod = reader.open(path, remove_tmp=True)

    # Get item
    item = prod.stac.create_item()

    # Add item to catalogue
    catalog.add_item(item)

2025-12-22 17:27:04,073 - [INFO] - *** LC08_L1TP_200030_20201220_20210310_02_T1.tar ***
'_vectorize' function is not lazy yet. Computing the raster.
2025-12-22 17:27:06,514 - [DEBUG] - Compute footprint for STAC Item for 20201220T104856_L8_200030_OLI_TIRS
2025-12-22 17:27:06,516 - [DEBUG] - Compute extent for STAC Item for 20201220T104856_L8_200030_OLI_TIRS
2025-12-22 17:27:06,517 - [DEBUG] - Creating STAC Item for 20201220T104856_L8_200030_OLI_TIRS
2025-12-22 17:27:06,517 - [DEBUG] - Add quicklook to STAC Item for 20201220T104856_L8_200030_OLI_TIRS
2025-12-22 17:27:06,518 - [WARNING] - No quicklook found in 20201220T104856_L8_200030_OLI_TIRS
2025-12-22 17:27:06,519 - [DEBUG] - Add EO extension to STAC Item for 20201220T104856_L8_200030_OLI_TIRS
2025-12-22 17:27:06,526 - [DEBUG] - Add PROJ extension to STAC Item for 20201220T104856_L8_200030_OLI_TIRS
2025-12-22 17:27:06,527 - [DEBUG] - Add VIEW extension to STAC Item for 20201220T104856_L8_200030_OLI_TIRS
2025-12-22 17:27:06,528 - [DEB

In [7]:
# Save catalog
catalog.describe()
catalog.normalize_and_save(tmp.name, catalog_type=pystac.CatalogType.SELF_CONTAINED)

* <Catalog id=SERTIT_101>
  * <Item id=20201220T104856_L8_200030_OLI_TIRS>
  * <Item id=20111110T103612_L5_200030_TM>
  * <Item id=20191215T110441_S2_T30TXP_L1C_114155>


In [8]:
list(catalog.get_items())[0]

<Item id=20201220T104856_L8_200030_OLI_TIRS>

# Query the catalog

`EODAG` is an opensource python library that implements STAC and allows you to query your local STAC catalog.  
Look at [here](https://eodag.readthedocs.io/en/stable/notebooks/tutos/tuto_stac_client.html) for a detailed tutorial.

In [9]:
# Create an EODAG custom STAC provider
dag = EODataAccessGateway()

# Set EODAG logging level to WARNING
setup_logging(verbose=1)

# Add the custom STAC provider, exactly like in the tutorial mentioned above
dag.update_providers_config("""
stac_http_provider:
    search:
        type: StaticStacSearch
        api_endpoint: %s
    products:
        GENERIC_PRODUCT_TYPE:
            productType: '{productType}'
    download:
        type: HTTPDownload
        base_uri: %s
        flatten_top_dirs: True
        outputs_prefix: %s
""" % (catalog_path, tmp.name, tmp.name))

# Set the custom STAC provider as preferred
dag.set_preferred_provider("stac_http_provider")

In [10]:
# Query every product from inside the catalog
all_products = dag.search()

In [11]:
# Load an AOI
aoi_path = os.path.join("/home", "aois", "DAX.geojson")
aoi = gpd.read_file(aoi_path)
aoi_geojson = mapping(aoi.geometry.values[0])

# Query spatially with the AOI and temporally with a time period
query_args = {"start": "2020-05-01", "end": "2022-05-06", "geom": aoi.geometry.values[0]}
query_products = dag.search(**query_args)

In [12]:
query_products[0]

EOProduct(id=LC08_L1GT_105168_20200501_20200820_02_T2, provider=usgs_satapi_aws)

In [13]:
query_products[0].assets

{'ANG.txt': {'title': 'ANG.txt', 'description': 'Collection 2 Level-1 Angle Coefficients File (ANG)', 'type': 'text/plain', 'roles': ['auxiliary'], 'href': 's3://usgs-landsat/collection02/level-1/standard/oli-tirs/2020/105/168/LC08_L1GT_105168_20200501_20200820_02_T2/LC08_L1GT_105168_20200501_20200820_02_T2_ANG.txt', 'file:checksum': '134042105fa9f08f9af5d399f53014782242e0df0c4ac37c3c0bfeef51495da796e7f0ffe7a85ffbb36f28b81c44710bad77b6c63d9ffb3a559d894f80ba94f2db48', 'storage:platform': 'AWS', 'storage:requester_pays': True}, 'B1.TIF': {'title': 'B1.TIF', 'description': 'Collection 2 Level-1 Coastal/Aerosol Band (B1) Top of Atmosphere Radiance', 'type': 'image/vnd.stac.geotiff; cloud-optimized=true', 'roles': ['data'], 'eo:bands': [{'name': 'B1', 'common_name': 'coastal', 'gsd': 30, 'center_wavelength': 0.44}], 'href': 's3://usgs-landsat/collection02/level-1/standard/oli-tirs/2020/105/168/LC08_L1GT_105168_20200501_20200820_02_T2/LC08_L1GT_105168_20200501_20200820_02_T2_B1.TIF', 'file:checksum': '1340a4d85cbfc9e749868b39f5dd68013ecc017213c3f59293b9e92ae587e1f5ef59e1f3e7d2f26ccb1146ab889d19d7acecc526bc041310ab8e5c2541683bd67447', 'storage:platform': 'AWS', 'storage:requester_pays': True}, 'B10.TIF': {'title': 'B10.TIF', 'description': 'Collection 2 Level-1 Thermal Infrared Band 10.9 (B10) Top of Atmosphere Radiance', 'type': 'image/vnd.stac.geotiff; cloud-optimized=true', 'roles': ['data'], 'eo:bands': [{'name': 'B10', 'common_name': 'lwir11', 'gsd': 100, 'center_wavelength': 10.9}], 'href': 's3://usgs-landsat/collection02/level-1/standard/oli-tirs/2020/105/168/LC08_L1GT_105168_20200501_20200820_02_T2/LC08_L1GT_105168_20200501_20200820_02_T2_B10.TIF', 'file:checksum': '134041c33e87ab088d1b34cd19ea92fc6ad8ef9349ce06fae187b8b3a6a479ecf552ec4e7c4e6aa5f5f46578a22603c613836cbb3cc4cd8de228fd09df48b2623c66', 'storage:platform': 'AWS', 'storage:requester_pays': True}, 'B11.TIF': {'title': 'B11.TIF', 'description': 'Collection 2 Level-1 Thermal Infrared Band 12.0 (B11) Top of Atmosphere Radiance', 'type': 'image/vnd.stac.geotiff; cloud-optimized=true', 'roles': ['data'], 'eo:bands': [{'name': 'B11', 'common_name': 'lwir12', 'gsd': 100, 'center_wavelength': 12.01}], 'href': 's3://usgs-landsat/collection02/level-1/standard/oli-tirs/2020/105/168/LC08_L1GT_105168_20200501_20200820_02_T2/LC08_L1GT_105168_20200501_20200820_02_T2_B11.TIF', 'file:checksum': '1340a81b970a82ef597dcb2d7f0590c292069ed1fa22327e9dcb2b2c1fb3fcd40538b41517cec318ac3f2bf5b4b797d7db3bdf837f5273ca956ddd456198bc747da6', 'storage:platform': 'AWS', 'storage:requester_pays': True}, 'B2.TIF': {'title': 'B2.TIF', 'description': 'Collection 2 Level-1 Blue Band (B2) Top of Atmosphere Radiance', 'type': 'image/vnd.stac.geotiff; cloud-optimized=true', 'roles': ['data'], 'eo:bands': [{'name': 'B2', 'common_name': 'blue', 'gsd': 30, 'center_wavelength': 0.48}], 'href': 's3://usgs-landsat/collection02/level-1/standard/oli-tirs/2020/105/168/LC08_L1GT_105168_20200501_20200820_02_T2/LC08_L1GT_105168_20200501_20200820_02_T2_B2.TIF', 'file:checksum': '1340133f19a44d017bed9af7af9d07fbd2575b4bd74f0c55d2fd527e6b9cb3aa27c8bda538749cba31d7c281e62a1fc71aff127ede354bc66503ed6d7261330ab4e9', 'storage:platform': 'AWS', 'storage:requester_pays': True}, 'B3.TIF': {'title': 'B3.TIF', 'description': 'Collection 2 Level-1 Green Band (B3) Top of Atmosphere Radiance', 'type': 'image/vnd.stac.geotiff; cloud-optimized=true', 'roles': ['data'], 'eo:bands': [{'name': 'B3', 'common_name': 'green', 'gsd': 30, 'center_wavelength': 0.56}], 'href': 's3://usgs-landsat/collection02/level-1/standard/oli-tirs/2020/105/168/LC08_L1GT_105168_20200501_20200820_02_T2/LC08_L1GT_105168_20200501_20200820_02_T2_B3.TIF', 'file:checksum': '1340521acf8aab52a6f233669e24049878536a4c9c203eaef6f91b7e9d3e9d1b4c1991ba5f1d6897305e5abb1282475e6881b6851d56204658afb6a6e0347d389a80', 'storage:platform': 'AWS', 'storage:requester_pays': True}, 'B4.TIF': {'title': 'B4.TIF', 'description': 'Collection 2 Level-1 Red Band (B4) Top of Atmosphere Ra

## Display the results

We can use `folium` to display the results geometry over a map.

In [14]:
import folium

# Create a map zoomed over the search area
fmap = folium.Map((43.2, -1.05), zoom_start=7)

# Add a layer green layer for the query over the AOI
folium.GeoJson(
    data=all_products.as_geojson_object(),
    tooltip = "All products stored in the catalog",
    style_function=lambda x: {'color': 'green'}
).add_to(fmap)

# Add a layer green layer for the query over the AOI
folium.GeoJson(
    data=query_products.as_geojson_object(),
    tooltip = "Retrieved products with the query",
    style_function=lambda x: {'color': 'red'}
).add_to(fmap)

# Add a layer blue layer for the AOI
folium.GeoJson(
    data=aoi_geojson,
    tooltip = "DAX AOI",
    style_function=lambda x: {'color': 'blue'}
).add_to(fmap)

fmap

In [15]:
# Clean the tmp directory
tmp.cleanup()